In [3]:
import pandas as pd

# Data Filter

In [9]:
df = pd.read_csv('2022-09-19~辟谣~博文详情.csv')

C:\Users\qqcom\AppData\Local\Temp\ipykernel_17872\3326995999.py:1: DtypeWarning: Columns (6,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2022-09-19~辟谣~博文详情.csv')


In [10]:
df.columns

Index(['博文ID', '博文链接', '博文内容', '博文关联话题', '博文发布时间', '博文图片链接', '博文转发量', '博文评论量',
       '博文点赞量', '博主ID', '博主名称', '博主链接', '博主图像', '博主简介', '博主关注数', '博主粉丝数',
       '博主博文数', '博主性别', '博主认证信息', '原文博文ID', '原文博文链接', '原文博文内容', '原文博文关联话题',
       '原文博文发布时间', '原文博文图片链接', '原文博文转发量', '原文博文评论量', '原文博文点赞量', '原文博主ID',
       '原文博主名称', '原文博主链接', '原文博主图像', '原文博主简介', '原文博主关注数', '原文博主粉丝数', '原文博主博文数',
       '原文博主性别', '原文博主认证信息'],
      dtype='object')

In [11]:
df.shape

(1752650, 38)

In [12]:
df['原文博文发布时间'] = pd.to_datetime(df['原文博文发布时间'])
start_date = pd.to_datetime('2019-12-01')
df = df[(df['原文博文发布时间'] >= start_date)]
df.shape

(867986, 38)

In [13]:
#Delete the value whose original content is empty
df = df.dropna(subset=['原文博文内容']) 

# Filter out the data related to COVID-19 in the original content of Weibo.
wordlist_file = 'covid_related_words.txt'
with open(wordlist_file, 'r', encoding='utf-8') as file:
    covid_wordlist = [word.strip() for word in file.readlines()] #.strip()移除首尾字符

mask = df['原文博文内容'].str.contains('|'.join(covid_wordlist), case=False)
filtered_df = df[mask]

filtered_df.shape

(91002, 38)

In [14]:
filtered_df.to_csv('filtered.csv', encoding='utf-8-sig')

### Use ChatGPT to get train dataset (n=1000)

In [15]:
from openai import OpenAI
import openai
import pandas as pd
import json

In [16]:
df = pd.read_csv('filtered.csv', nrows=1000)

In [38]:
api_key = 'sk-YWes16xh9fuURVQM47D0B14eEb5a4c30B29c4d4d32F86731'
base_url = "https://api.kwwai.top"
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

In [39]:
for index, row in df.iterrows():
    text = row['原文博文内容']
    prompt_chi = (f"""{text}
                上述内容来自微博新冠疫情辟谣相关帖子,请判断这段内容是否属于辟谣信息：
                0: 否
                1: 是
                分类结果只输出0或1,并说明原因,原因不超过30个字,结果用json格式输出,并包含两个key值:classify,reason
                """)
    response = client.completions.create(
                model="gpt-3.5-turbo-instruct",
                prompt=  prompt_chi,
                temperature=0,
                max_tokens=200
                )
    print(response)
    response = response.choices[0].text.strip()
    try:
        data = json.loads(response)
        classify = data['classify']
        reason = data['reason']
        df.at[index, 'classify'] = classify
        df.at[index, 'reason'] = reason
    except:
        print("Wrong output!")
        continue

df.head(2)
df.shape

<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="logo.png"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#ffffff"/><meta name="description" content="OpenAI 接口聚合管理，支持多种渠道包括 Azure，可用于二次分发管理 key，仅单可执行文件，已打包好 Docker 镜像，一键部署，开箱即用"/><title>New API</title><script defer="defer" src="/static/js/main.461fb036.js"></script><link href="/static/css/main.28d1d0b7.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div></body></html>


AttributeError: 'str' object has no attribute 'choices'

In [ ]:
df.to_csv('gpt_result_train.csv', encoding='utf-8-sig')

### 朴素贝叶斯文本分类

#### Segment

In [25]:
import jieba
import re

In [26]:
df_gpt_result = pd.read_csv('gpt_result_train.csv')
df = df_gpt_result[['原文博文内容', 'classify']]

In [27]:
#读取停用词
stopwords = [line.strip() for line in open('C:/D盘/HK/covid-19 rumor/stopwords-master/stopwords-master/cn_stopwords.txt',encoding='UTF-8').readlines()]

In [ ]:
#分词
def seg_depart(sentence):
    pattern = '[\u4E00-\u9FFF]+' #保留中文字符
    ch_str =re.findall(pattern, str(sentence).strip()) #.strip()去除行末换行符
    sentence_depart = jieba.cut(str(ch_str))  
    outstr = ''
    for word in sentence_depart:
        if word not in stopwords:  #去除停用词
            if word != '\t':
                if len(word)>=2:
                    outstr += word
                    outstr += ' '
    outstr = outstr[:-1]
    return outstr

df['cut'] = df['原文博文内容'].apply(seg_depart)

C:\Users\qqcom\AppData\Local\Temp\ipykernel_14512\2934482835.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cut'] = df['原文博文内容'].apply(seg_depart)


In [ ]:
df.head(2)

,原文博文内容,classify,cut
0,【#美传染病专家赴印度感染新冠去世#，此前已注射两针疫苗】5月3日，美国HIV/AIDS总统...,1.0,传染病 专家 印度 感染 新冠 去世 此前 注射 两针 疫苗 美国 总统 顾问 委员会 成员...
1,朱易#朱易女单自由滑##朱易冬奥会首秀##朱易# #冬奥花滑女单名额选拔真相# #朱易花滑女...,1.0,朱易 朱易 女单 自由滑 朱易 冬奥会 首秀 朱易 冬奥 花滑 女单 名额 选拔 真相 朱易...


#### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer(max_df = 0.8,  #超过这一占比的词太频繁，去除
                       min_df = 5,    #低于5次的词太独特，去除
                        )
feature_names_df = pd.DataFrame(vect.fit_transform(df['cut']).toarray(), columns=vect.get_feature_names_out())
feature_names_df

,一一,一个月,一事,一人,一件,一份,一位,一例,一半,一句,...,鸡蛋,鹤岗,麻烦,黄冈,黄河,黄浦区,黄腔,黑龙江,鼓励,鼻咽癌
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = feature_names_df
y = df.classify

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


#### Train Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB #朴素贝叶斯

In [ ]:
nb = MultinomialNB()

nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
recall = recall_score(y_test, predictions)
print("召回率:", recall)

accuracy = accuracy_score(y_test, predictions)
print("准确率:", accuracy)

召回率: 0.964824120603015
准确率: 0.96


#### Prediction

In [ ]:
df_unkown = pd.read_csv('filtered.csv')
df_unkown['cut'] = df_unkown['原文博文内容'].apply(seg_depart)
X_unknown = vect.transform(df_unkown['cut'])
predictions = nb.predict(X_unknown)
df_unkown['pred_classify'] = predictions
df_final = df_unkown[df_unkown['pred_classify'] == 1]

C:\Users\qqcom\AppData\Local\Temp\ipykernel_14512\580793033.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unkown = pd.read_csv('filtered.csv')


In [ ]:
df_final.shape

(88636, 41)

In [ ]:
df_final.to_csv('original_data.csv', encoding='utf-8-sig')

### Keep data after 2019-12-01

In [44]:
df = pd.read_csv('original_data.csv')
df['原文博文发布时间'] = pd.to_datetime(df['原文博文发布时间'])
start_date = pd.to_datetime('2019-12-01')
df_filtered = df[(df['原文博文发布时间'] >= start_date)]
df_filtered.shape

C:\Users\qqcom\AppData\Local\Temp\ipykernel_17872\896485550.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('original_data.csv')


(88165, 43)

In [45]:
df_filtered.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', '博文ID', '博文链接', '博文内容',
       '博文关联话题', '博文发布时间', '博文图片链接', '博文转发量', '博文评论量', '博文点赞量', '博主ID', '博主名称',
       '博主链接', '博主图像', '博主简介', '博主关注数', '博主粉丝数', '博主博文数', '博主性别', '博主认证信息',
       '原文博文ID', '原文博文链接', '原文博文内容', '原文博文关联话题', '原文博文发布时间', '原文博文图片链接',
       '原文博文转发量', '原文博文评论量', '原文博文点赞量', '原文博主ID', '原文博主名称', '原文博主链接', '原文博主图像',
       '原文博主简介', '原文博主关注数', '原文博主粉丝数', '原文博主博文数', '原文博主性别', '原文博主认证信息',
       'pred_classify', 'cut'],
      dtype='object')

In [46]:
df_filtered = df_filtered.drop(df_filtered.columns[:3],axis=1)
df_filtered.columns

Index(['博文ID', '博文链接', '博文内容', '博文关联话题', '博文发布时间', '博文图片链接', '博文转发量', '博文评论量',
       '博文点赞量', '博主ID', '博主名称', '博主链接', '博主图像', '博主简介', '博主关注数', '博主粉丝数',
       '博主博文数', '博主性别', '博主认证信息', '原文博文ID', '原文博文链接', '原文博文内容', '原文博文关联话题',
       '原文博文发布时间', '原文博文图片链接', '原文博文转发量', '原文博文评论量', '原文博文点赞量', '原文博主ID',
       '原文博主名称', '原文博主链接', '原文博主图像', '原文博主简介', '原文博主关注数', '原文博主粉丝数', '原文博主博文数',
       '原文博主性别', '原文博主认证信息', 'pred_classify', 'cut'],
      dtype='object')

In [47]:
df_filtered.shape

(88165, 40)

In [49]:
df_filtered.to_csv('original_data.csv', encoding='utf-8-sig',index=False)